In [1]:
# https://medium.com/analytics-vidhya/demonstrating-calculation-of-tf-idf-from-sklearn-4f9526e7e78b

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd



In [2]:
# load all the documents in the context corpus
# should be array, where each doc should be one array.
# Load the dataframe consisting of the data
df_train = pd.read_pickle("./data_frames/df_train.pkl")
df_train.head()
# columns: context_parsed (array where each sentence is a string)
# should use lemmas?

,choices,context,question,correct_answer,correct_answer_parsed,correct_answer_parsed_tok,correct_answer_parsed_stop,correct_answer_parsed_lemma,correct_answer_parsed_lemma_stop,correct_answer_loc,...,answer_location,sent_with_ans,answer_locations_text,paragraph_len,loc_frac,context_parsed,context_parsed_tok_lemma,context_parsed_tok_lemma_stop,context_parsed_tok_stop,context_parsed_tok
0,"[{'end': 224, 'extra': None, 'start': 175, 'te...",Visum för affärs- och konferensbesök\r\nOm du ...,Vad är ett visum?,ett tillstånd för att resa in och vara i ett land,ett tillstånd för att resa in och vara i ett land,"[ett, tillstånd, för, att, resa, in, och, vara...","[tillstånd, resa, in, land]","[en, tillstånd, för, att, resa, in, och, vara,...","[tillstånd, resa, in, land]",175,...,2,visum är ett tillstånd för att resa in och var...,[2],21,0.095238,"[visum för affärs- och konferensbesök, om du s...","[[visa, för, affär, och, konferensbesök], [om,...","[[visa, affär, konferensbesök], [om, skola, be...","[[visum, affärs-, konferensbesök], [om, ska, b...","[[visum, för, affärs-, och, konferensbesök], [..."
1,"[{'end': 707, 'extra': None, 'start': 681, 'te...",Eget företag\r\nEfter beslut\r\nBeslutet skick...,Vad ska du ta med när du hämtar ditt beslut?,ditt pass,ditt pass,"[ditt, pass]",[pass],"[du, pass]",[pass],156,...,2,när du ska hämta ditt beslut ska du ta med dit...,"[2, 3]",19,0.105263,"[eget företag, efter beslut beslutet skickas t...","[[egen, företag], [efter, beslut, beslut, skic...","[[egen, företag], [efter, beslut, beslut, skic...","[[eget, företag], [efter, beslut, beslutet, sk...","[[eget, företag], [efter, beslut, beslutet, sk..."
2,"[{'end': 1165, 'extra': None, 'start': 1154, '...",Utbytesdoktorander\r\nIntervju\r\nDu kommer at...,Hur länge gäller ett uppehållstillstånd för be...,ett år,ett år,"[ett, år]",[år],"[en, år]",[år],1619,...,18,så länge gäller tillståndet ett uppehållstills...,[18],25,0.720000,"[utbytesdoktorander intervju, du kommer att få...","[[utbytesdoktorand, intervju], [du, komma, att...","[[utbytesdoktorand, intervju], [du, komma, få,...","[[utbytesdoktorander, intervju], [du, kommer, ...","[[utbytesdoktorander, intervju], [du, kommer, ..."
3,"[{'end': 598, 'extra': None, 'start': 589, 'te...",Eget företag\r\nNär vi har tagit emot din ansö...,Vad är ett uppehållstillståndskort?,ett bevis,ett bevis,"[ett, bevis]",[bevis],"[en, bevis]",[bevis],589,...,8,kortet är ett bevis på att du har tillstånd at...,[8],21,0.380952,"[eget företag, när vi har tagit emot din ansök...","[[egen, företag], [när, vi, ha, ta, emot, du, ...","[[egen, företag], [när, ta, emot, ansökan], [o...","[[eget, företag], [när, tagit, emot, ansökan],...","[[eget, företag], [när, vi, har, tagit, emot, ..."
4,"[{'end': 1932, 'extra': None, 'start': 1924, '...",Flytta tillbaka till Sverige\r\nSå ansöker du\...,Vad är ett uppehållstillståndskort?,ett bevis,ett bevis,"[ett, bevis]",[bevis],"[en, bevis]",[bevis],673,...,8,kortet är ett bevis på att du har tillstånd at...,[8],21,0.380952,"[flytta tillbaka till sverige, så ansöker du d...","[[flytta, tillbaka, till, Sverige], [så, ansök...","[[flytta, tillbaka, Sverige], [så, ansöka, du,...","[[flytta, tillbaka, sverige], [så, ansöker, du...","[[flytta, tillbaka, till, sverige], [så, ansök..."


In [5]:
def combine_docs(df, col):
    docs = []
    for index, row in df.iterrows():
        context = row[col]
        if isinstance(context[0], list):
            sents = []
            for sent in context:
                sent_str = ' '.join(sent)
                sents.append(sent_str)
            context = sents
        doc = ' '.join(context)
        docs.append(doc)
    return docs

In [11]:
tfidf_transformer = TfidfTransformer()
cv = CountVectorizer()

In [12]:
def create_tf_df(docs):
    word_count_vector = cv.fit_transform(docs)
    tf = pd.DataFrame(word_count_vector.toarray(), columns=cv.get_feature_names())
    return word_count_vector, tf


In [13]:
tfidf_transformer = TfidfTransformer()
def create_idf_df(word_count_vector):
    X = tfidf_transformer.fit_transform(word_count_vector)
    idf = pd.DataFrame({'feature_name':cv.get_feature_names(), 'idf_weights':tfidf_transformer.idf_})
    print(idf)
    return X, idf

In [16]:
docs = combine_docs(df_train, 'context_parsed')
word_count_vector, tf = create_tf_df(docs)
X, idf = create_idf_df(word_count_vector)
tf_idf = pd.DataFrame(X.toarray() ,columns=cv.get_feature_names())
print(tf_idf)

            feature_name  idf_weights
0                    000     4.651178
1                   0000     6.078294
2                    010     6.483759
3                    011     6.483759
4                   0128     6.078294
...                  ...          ...
14994     övningskörning     6.771441
14995  övningsverksamhet     6.260615
14996              övrig     4.925614
14997             övriga     3.057869
14998             övrigt     5.567468

[14999 rows x 2 columns]
          000  0000  010  011  0128   03   05  0630   07  0730  ...  \
0    0.051548   0.0  0.0  0.0   0.0  0.0  0.0   0.0  0.0   0.0  ...   
1    0.000000   0.0  0.0  0.0   0.0  0.0  0.0   0.0  0.0   0.0  ...   
2    0.000000   0.0  0.0  0.0   0.0  0.0  0.0   0.0  0.0   0.0  ...   
3    0.000000   0.0  0.0  0.0   0.0  0.0  0.0   0.0  0.0   0.0  ...   
4    0.000000   0.0  0.0  0.0   0.0  0.0  0.0   0.0  0.0   0.0  ...   
..        ...   ...  ...  ...   ...  ...  ...   ...  ...   ...  ...   
957  0.000000   0.0  

/Users/filippakarrfelt/opt/anaconda3/envs/dp/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/filippakarrfelt/opt/anaconda3/envs/dp/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [17]:
docs = combine_docs(df_train, 'context_parsed_tok_lemma_stop')
word_count_vector_lemma_stop, tf_lemma_stop = create_tf_df(docs)
X_lemma_stop, idf = create_idf_df(word_count_vector_lemma_stop)
tf_idf_lemma_stop = pd.DataFrame(X_lemma_stop.toarray() ,columns=cv.get_feature_names())
print(tf_idf_lemma_stop)

            feature_name  idf_weights
0                    000     4.651178
1                   0000     6.078294
2                    010     6.483759
3                    011     6.483759
4                   0128     6.078294
...                  ...          ...
10707  övningsanläggning     6.078294
10708     övningskörning     6.771441
10709  övningsverksamhet     6.260615
10710              övrig     2.935579
10711             övrigt     5.567468

[10712 rows x 2 columns]
         000  0000  010  011  0128  01454   03   05   06  06975  ...  \
0    0.06595   0.0  0.0  0.0   0.0    0.0  0.0  0.0  0.0    0.0  ...   
1    0.00000   0.0  0.0  0.0   0.0    0.0  0.0  0.0  0.0    0.0  ...   
2    0.00000   0.0  0.0  0.0   0.0    0.0  0.0  0.0  0.0    0.0  ...   
3    0.00000   0.0  0.0  0.0   0.0    0.0  0.0  0.0  0.0    0.0  ...   
4    0.00000   0.0  0.0  0.0   0.0    0.0  0.0  0.0  0.0    0.0  ...   
..       ...   ...  ...  ...   ...    ...  ...  ...  ...    ...  ...   
957  0.00000  

/Users/filippakarrfelt/opt/anaconda3/envs/dp/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/filippakarrfelt/opt/anaconda3/envs/dp/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [18]:
# save tf-idf values
tf_idf.to_pickle("./data_frames/tf-idf/df_tf_idf.pkl")
tf_idf_lemma_stop.to_pickle("./data_frames/tf-idf/df_tf_idf_lemma_stop.pkl")